# Tangram Analysis: 2D Coordinates Recovery of Single Cells

## Xenium Breast Cancer, InSitu Replicate 1

https://www.10xgenomics.com/products/xenium-in-situ/human-breast-dataset-explorer



### 1. Import packages

In [60]:
import os,csv,re
import pandas as pd
import numpy as np

import math
from sklearn.cluster import KMeans

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import pickle

#Read original data and save it to h5ad
import scanpy as sc
from scanpy import read_10x_h5
from scanpy import read_h5ad

import tangram as tg

In [2]:
tg.__version__

'1.0.3'

In [ ]:
import sys
sys.path.append('/lustre03/project/6075067/calcium/2021/CeLEry/CeLEry_package')

import CeLEry as cel

In [4]:
def make_annData_spatial (adata, spatial, min_cells = 3, filtered = False):
    """ 
    adata: an annData file for the transcriptomics data
    spatial: an pandas dataframe recording the location information for each spot
    """
    if filtered == False:
        adata.obs["select"] = spatial[1]
        adata.obs["x_cord"] = spatial[2]
        adata.obs["y_cord"] = spatial[3]
        adata.obs["x_pixel"] = spatial[4]
        adata.obs["y_pixel"] = spatial[5]
        # Select captured samples
        adata = adata[adata.obs["select"] == 1]
    else:
        spatialsub = spatial[spatial.iloc[:,0] == 1]
        adata.obs = adata.obs.join(spatialsub)
        adata.obs.columns = ['select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel']
    adata.var_names = [i.upper() for i in list(adata.var_names)]
    adata.var["genename"] = adata.var.index.astype("str")
    #
    adata.var_names_make_unique()
    prefilter_genes(adata, min_cells=min_cells) # avoiding all genes are zeros
    prefilter_specialgenes(adata)
    #Normalize and take log for UMI
    #sc.pp.normalize_per_cell(adata)
    #sc.pp.log1p(adata)
    return adata

In [5]:
def prefilter_genes(adata,min_counts=None,max_counts=None,min_cells=10,max_cells=None):
    if min_cells is None and min_counts is None and max_cells is None and max_counts is None:
        raise ValueError('Provide one of min_counts, min_genes, max_counts or max_genes.')
    id_tmp=np.asarray([True]*adata.shape[1],dtype=bool)
    id_tmp=np.logical_and(id_tmp,sc.pp.filter_genes(adata.X,min_cells=min_cells)[0]) if min_cells is not None  else id_tmp
    id_tmp=np.logical_and(id_tmp,sc.pp.filter_genes(adata.X,max_cells=max_cells)[0]) if max_cells is not None  else id_tmp
    id_tmp=np.logical_and(id_tmp,sc.pp.filter_genes(adata.X,min_counts=min_counts)[0]) if min_counts is not None  else id_tmp
    id_tmp=np.logical_and(id_tmp,sc.pp.filter_genes(adata.X,max_counts=max_counts)[0]) if max_counts is not None  else id_tmp
    adata._inplace_subset_var(id_tmp)

In [6]:
def prefilter_specialgenes(adata,Gene1Pattern="ERCC",Gene2Pattern="MT-"):
    id_tmp1=np.asarray([not str(name).startswith(Gene1Pattern) for name in adata.var_names],dtype=bool)
    id_tmp2=np.asarray([not str(name).startswith(Gene2Pattern) for name in adata.var_names],dtype=bool)
    id_tmp=np.logical_and(id_tmp1,id_tmp2)
    adata._inplace_subset_var(id_tmp)

### 2. Load data:

In [7]:
# cells to be filtered out based on total UMI and number of genes expressed
os.chdir("")
lowUMI_cellID = pd.read_csv("cell_ID_toRemove_filtered_75_25.csv",sep=",",na_filter=False,index_col=0)

In [8]:
#Read in gene expression and spatial location
os.chdir("")
adata = read_10x_h5("Xenium_FFPE_Human_Breast_Cancer_Rep1_cell_feature_matrix.h5")
spatial_full = pd.read_csv("Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv",sep=",",na_filter=False,index_col=0)

In [9]:
os.chdir("") #scheme 4

train_index = pd.read_csv("trainCell_index.csv",sep=",")
test_index = pd.read_csv("testCell_index.csv",sep=",")

train_index = list(train_index.iloc[:,1])
test_index = list(test_index.iloc[:,1])

In [14]:
spatial_full

,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,total_counts,cell_area,nucleus_area
cell_id,,,,,,,,
1,377.663005,843.541888,154,0,0,154,110.361875,45.562656
2,382.078658,858.944818,64,0,0,64,87.919219,24.248906
3,319.839529,869.196542,57,0,0,57,52.561875,23.526406
4,259.304707,851.797949,120,0,0,120,75.230312,35.176719
5,370.576291,865.193024,120,0,0,120,180.218594,34.499375
...,...,...,...,...,...,...,...,...
167778,7455.404785,5115.021094,238,1,0,239,219.956094,61.412500
167779,7483.771045,5111.720703,80,0,0,80,38.427969,25.964844
167780,7470.119580,5119.350366,406,0,0,406,287.690469,86.158125


In [15]:
# The pixel size of Xenium 0.2125 microns. 
# Coordinates in microns from cells.csv.gz can be converted to pixel coordinates 
# by dividing by the pixel size. The origin of the coordinate system is the upper left of the TIFF image.


pixel_size = 0.2125

spatial = pd.DataFrame()
spatial['0'] = spatial_full.x_centroid
spatial['1'] = [1] * 167782
spatial['2'] = spatial_full.x_centroid
spatial['3'] = spatial_full.y_centroid 
spatial['4'] = spatial_full.x_centroid / pixel_size
spatial['5'] = spatial_full.y_centroid / pixel_size

b = ['CellID '] * 167782
a = list(range(1,167783))

#spatial.index = [m+str(n) for m,n in zip(b,a)]

spatial.index = spatial.index.astype('str')


spatial = spatial.drop(['0'], axis = 1)
spatial.index.name = '0'
spatial.columns = spatial.columns.astype('int64')
spatial = spatial.astype('int64')

adata.obs_names = spatial.index.astype('str')

In [20]:
TrainDatafull = make_annData_spatial(adata.copy(), spatial, filtered = True)
TrainDatafull.var['genename'] = TrainDatafull.var.gene_ids

TrainDatafull

AnnData object with n_obs × n_vars = 167782 × 313
    obs: 'select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'

### 3. Filter out cells with low UMI and low number of genes expressed

In [25]:
os.chdir("")
TrainDatafull.obs_names = TrainDatafull.obs_names.astype(np.int64) 
TrainDatafull_filtered = TrainDatafull.copy()[~TrainDatafull.obs_names.isin(lowUMI_cellID.x), :]
TrainDatafull_filtered

View of AnnData object with n_obs × n_vars = 42228 × 313
    obs: 'select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'

In [27]:
sc.pp.normalize_total(TrainDatafull_filtered)

### 4. Split data into train and test data 

In [30]:
### Spliting into training and testing data for prediction/evalutaiton:

# split the data into training and testing data (90% train, 10% test)
# making sure to use same sets across all methods (see CelERY script for generating random splits of the test/train data)

train_index = np.array(train_index).astype('str')
test_index = np.array(test_index).astype('str')

# 90% train
DataSubtrain90_coor = TrainDatafull_filtered[train_index,]

# 10% holdoff
DataSubtest10_coor = TrainDatafull_filtered[test_index,]


In [31]:
DataSubtrain90_coor

View of AnnData object with n_obs × n_vars = 38006 × 313
    obs: 'select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'

In [32]:
DataSubtest10_coor

View of AnnData object with n_obs × n_vars = 4222 × 313
    obs: 'select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel'
    var: 'gene_ids', 'feature_types', 'genome', 'genename'

In [33]:
holdoff = 10
dataSection1 = DataSubtrain90_coor 
dataSection2 = DataSubtest10_coor

In [34]:
dataSection1.obs

,select,x_cord,y_cord,x_pixel,y_pixel
0,,,,,
7,1,296,856,1394,4031
32,1,277,856,1306,4029
54,1,182,852,856,4013
55,1,132,864,623,4069
59,1,148,843,696,3968
...,...,...,...,...,...
167761,1,7486,5212,35231,24531
167771,1,7458,5095,35099,23977
167772,1,7469,5093,35148,23970


In [35]:
random.seed(2021)
torch.manual_seed(2021)
np.random.seed(2021)
## For each cell  of the "scRNA", find the predicted location on Spatial map
S1_xmax = dataSection1.obs['x_cord'].max() + 1
S1_xmin = dataSection1.obs['x_cord'].min() - 1
S1_ymax = dataSection1.obs['y_cord'].max() + 1
S1_ymin = dataSection1.obs['y_cord'].min() - 1
#
S2_xmax = dataSection2.obs['x_cord'].max() + 1
S2_xmin = dataSection2.obs['x_cord'].min() - 1
S2_ymax = dataSection2.obs['y_cord'].max() + 1
S2_ymin = dataSection2.obs['y_cord'].min() - 1


In [36]:
dataSection1.obs['x'] = dataSection1.obs['x_cord']
dataSection1.obs['y'] = dataSection1.obs['y_cord']

dataSection2.obs['x'] = dataSection2.obs['x_cord']
dataSection2.obs['y'] = dataSection2.obs['y_cord']

In [37]:
dataSection1.X = dataSection1.X.toarray()
dataSection2.X = dataSection2.X.toarray()

### 5. Run Tangram

In [38]:
## Map Section 2 data into Section 1 data
tg.pp_adatas(dataSection2, dataSection1, genes=None) #genes = 'None' uses all overlapping genes

INFO:root:313 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:313 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [39]:
dataSection1

AnnData object with n_obs × n_vars = 38006 × 313
    obs: 'select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel', 'x', 'y', 'uniform_density', 'rna_count_based_density'
    var: 'gene_ids', 'feature_types', 'genome', 'genename', 'n_cells'
    uns: 'training_genes', 'overlap_genes'

In [40]:
dataSection2

AnnData object with n_obs × n_vars = 4222 × 313
    obs: 'select', 'x_cord', 'y_cord', 'x_pixel', 'y_pixel', 'x', 'y'
    var: 'gene_ids', 'feature_types', 'genome', 'genename', 'n_cells'
    uns: 'training_genes', 'overlap_genes'

In [41]:
# remove extra data stored 

del adata
del TrainDatafull_filtered
del TrainDatafull

In [42]:
my_map = tg.map_cells_to_space(dataSection2, dataSection1, device='cpu')

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 313 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.370, KL reg: 0.000
Score: 0.812, KL reg: 0.001
Score: 0.828, KL reg: 0.001
Score: 0.833, KL reg: 0.001
Score: 0.835, KL reg: 0.001
Score: 0.836, KL reg: 0.001
Score: 0.837, KL reg: 0.001
Score: 0.838, KL reg: 0.001
Score: 0.838, KL reg: 0.001
Score: 0.839, KL reg: 0.001


INFO:root:Saving results..


In [43]:
my_map.X

array([[3.47447468e-07, 8.47554134e-08, 4.20595327e-08, ...,
        4.98230470e-08, 2.28130133e-07, 3.31897951e-07],
       [2.65370144e-08, 1.88062224e-08, 7.73048185e-08, ...,
        4.24695266e-08, 3.46192088e-08, 2.50979433e-08],
       [4.19942232e-08, 2.16521883e-07, 4.42786018e-07, ...,
        9.31636279e-08, 8.59325837e-08, 3.18603774e-08],
       ...,
       [9.09421427e-09, 5.65438647e-08, 1.32963223e-07, ...,
        1.19906517e-07, 2.15304198e-07, 1.49662313e-07],
       [1.08431792e-07, 5.47239019e-08, 3.32501138e-08, ...,
        1.57167094e-07, 1.45778714e-08, 1.08034705e-07],
       [8.47946410e-07, 4.36577807e-07, 3.32183163e-07, ...,
        3.01479076e-07, 3.90627122e-08, 3.79818289e-07]], dtype=float32)

In [45]:
my_map.write_h5ad("")


In [65]:
my_map.X.shape

(4222, 38006)

## Expected position:

In [68]:
location_sum = np.sum(my_map.X, axis=1)
location_pred_copy = my_map.X / location_sum.reshape(len(location_sum), 1)
pred_cord_transform = location_pred_copy.dot(np.array(dataSection1.obs[['x_cord', 'y_cord']]))

In [69]:
pred_cord_transform

array([[1890.31349158, 1991.05099866],
       [1767.19793308, 2810.41723462],
       [3467.17504933, 2294.02249064],
       ...,
       [3304.45497392, 2690.07885881],
       [2078.97792889, 2114.22753106],
       [2292.99099974, 2176.92317405]])

In [71]:
#pd.cor(pred_cord_transform[0:,], datatest.obs['x_cord'])

np.corrcoef(pred_cord_transform[:,0],dataSection2.obs['x_cord'])

array([[1.        , 0.76361513],
       [0.76361513, 1.        ]])

In [72]:
np.corrcoef(pred_cord_transform[:,1],dataSection2.obs['y_cord'])

array([[1.        , 0.51205401],
       [0.51205401, 1.        ]])